# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657244,35.2,1454474,77.7,1326117,70.9
Vcells,1220271,9.4,8388608,64.0,1975128,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [3]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 765179
PARAM$qsemillas <- 10

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [4]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  # Modified: Add the 'fold' column directly to the data.table
  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
  return(invisible(NULL)) # Return NULL invisibly as the data.table is modified in place
}

In [5]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}

In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}

In [7]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [8]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [9]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )

In [35]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("RefinedGridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("RefinedGridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 560

In [26]:
# data.table threads (0 = auto use all)
data.table::setDTthreads(0)
data.table::getDTthreads()

[1] 8

In [12]:
# Partition the dataset after loading and filtering
particionar(dataset,
  division = c(PARAM$training_pct, 100L - PARAM$training_pct),
  agrupa = "clase_ternaria",
  seed = PARAM$semilla_primigenia
)

In [13]:
# Overall counts per fold (1 = train, 2 = test)
fold_counts <- dataset[, .N, by = fold][order(fold)]
fold_counts[, pct := round(100 * N / sum(N), 2)]
fold_counts

# number of training rows (fold == 1)
n_train <- dataset[, sum(fold == 1L)]
n_train

fold,N,pct
<dbl>,<int>,<dbl>
1,115211,70
2,49385,30


[1] 115211

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [36]:
iter <- 0

cp_set        <- c(-0.5)
maxdepth_set  <- c(7, 8)
minsplit_set  <- c(60, 80, 100, 115, 130, 150, 180, 200, 250, 300, 400, 600, 800, 1000)
minbucket_set <- c(15, 20, 25, 29, 32, 38, 45, 50, 62, 250)


for (vmax_depth in maxdepth_set) {
  for (vmin_split in minsplit_set) {
    for (vmin_bucket in minbucket_set) {
      for (vcp in cp_set) {

        iter <- iter + 1
        cat(iter, " "); flush.console()
        if (iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

        param_basicos <- list(
          cp        = vcp,
          maxdepth  = vmax_depth,
          minsplit  = vmin_split,
          minbucket = vmin_bucket
        )

        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        tb_grid_search_detalle <- rbindlist(
          list(tb_grid_search_detalle,
               rbindlist(ganancias, use.names = TRUE, fill = TRUE)),
          use.names = TRUE, fill = TRUE
        )
        fwrite(tb_grid_search_detalle, "RefinedGridsearch_detalle.txt", sep = "\t")
      }
    }
  }
}


1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  

In [37]:
fwrite( tb_grid_search_detalle,
   file = "RefinedGridsearch_detalle.txt",
   sep = "\t"
)

In [38]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 600

In [39]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
439891,-0.1,6,20,10,43020000
644123,-0.1,6,20,10,47380000
213149,-0.1,6,20,10,45070000
366193,-0.1,6,20,10,46650000
303731,-0.1,6,20,10,44960000
439891,-1.0,6,20,10,43020000
644123,-1.0,6,20,10,47380000
213149,-1.0,6,20,10,45070000
366193,-1.0,6,20,10,46650000


In [40]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [41]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )

In [42]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-0.1,6,800,100,49812000,5
-1.0,6,800,100,49812000,5
-0.1,6,400,100,49496000,5
-1.0,6,400,100,49496000,5
-0.1,6,600,100,49470000,5
-1.0,6,600,100,49470000,5
-0.1,8,800,100,49410000,5
-1.0,8,800,100,49410000,5
-0.1,6,20,100,49264000,5


In [43]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "Refinedgridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )